# テストデータ生成のバグ修正

真っ黒な画面を学習してしまう箇所を修正した。

In [1]:
import os
import glob
import pandas as pd
import shutil
from datetime import datetime as dt
import datetime

import os
import shutil
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2
from tqdm import tqdm
import gc
import numpy as np
import hickle as hkl

import warnings
warnings.filterwarnings('ignore')

In [2]:
SAT_DIR = '../data/input/test/sat'
EXTERNAL_DIR = '../data/external'

In [3]:
test_info = '../data/input/inference_terms.csv'
test_info_df = pd.read_csv(test_info)
test_info_df.head()

,OpenData_96hr_Start,OpenData_96hr_End,Inference_24hr_Start,Inference_24hr_End,Evaluation_06hr,Evaluation_12hr,Evaluation_18hr,Evaluation_24hr
0,2018/1/2 16:00,2018/1/6 15:00,2018/1/6 16:00,2018/1/7 15:00,2018/1/6 21:00,2018/1/7 3:00,2018/1/7 9:00,2018/1/7 15:00
1,2018/1/9 16:00,2018/1/13 15:00,2018/1/13 16:00,2018/1/14 15:00,2018/1/13 21:00,2018/1/14 3:00,2018/1/14 9:00,2018/1/14 15:00
2,2018/1/16 16:00,2018/1/20 15:00,2018/1/20 16:00,2018/1/21 15:00,2018/1/20 21:00,2018/1/21 3:00,2018/1/21 9:00,2018/1/21 15:00
3,2018/1/23 16:00,2018/1/27 15:00,2018/1/27 16:00,2018/1/28 15:00,2018/1/27 21:00,2018/1/28 3:00,2018/1/28 9:00,2018/1/28 15:00
4,2018/1/30 16:00,2018/2/3 15:00,2018/2/3 16:00,2018/2/4 15:00,2018/2/3 21:00,2018/2/4 3:00,2018/2/4 9:00,2018/2/4 15:00


In [4]:
all_filepaths = []
for end_date in test_info_df.OpenData_96hr_End:
    filepaths = []
    end_date = dt.strptime(end_date, '%Y/%m/%d %H:%M')
    dir_name = end_date.strftime('%Y-%m-%d')
    filename = end_date.strftime('%Y-%m-%d-%H-%M')
    filepath = f'../data/input/test/sat/{dir_name}/{filename}.fv.png'
    for _ in range(0,25):
        filepaths.append(filepath)
    for t in range(1,24):
        
        date = end_date - datetime.timedelta(hours=t)
        dir_name = date.strftime('%Y-%m-%d')
        filename = date.strftime('%Y-%m-%d-%H-%M')

        filepath = f'../data/input/test/sat/{dir_name}/{filename}.fv.png'
        filepaths.append(filepath)
        filepaths = sorted(filepaths)
    all_filepaths += filepaths
all_filepaths

['../data/input/test/sat/2018-01-05/2018-01-05-16-00.fv.png',
 '../data/input/test/sat/2018-01-05/2018-01-05-17-00.fv.png',
 '../data/input/test/sat/2018-01-05/2018-01-05-18-00.fv.png',
 '../data/input/test/sat/2018-01-05/2018-01-05-19-00.fv.png',
 '../data/input/test/sat/2018-01-05/2018-01-05-20-00.fv.png',
 '../data/input/test/sat/2018-01-05/2018-01-05-21-00.fv.png',
 '../data/input/test/sat/2018-01-05/2018-01-05-22-00.fv.png',
 '../data/input/test/sat/2018-01-05/2018-01-05-23-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-00-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-01-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-02-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-03-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-04-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-05-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-06-00.fv.png',
 '../data/input/test/sat/2018-01-06/2018-01-06-07-00.fv.png',
 '../dat

In [5]:
minimizing_im_sz = 4
desired_im_sz = (512//minimizing_im_sz, 672//minimizing_im_sz) # (w, h)
DATA_DIR = '../data/features'

In [6]:
nt = 48
cur_loc = 0
test_starts = []
while cur_loc < len(all_filepaths) - nt + 1:
    test_starts.append(cur_loc)
    cur_loc += nt
test_starts
im_list = []
source_list = []
for loc in test_starts:
    source_list += [loc] * nt
print('Creating test data: ' + str(len(all_filepaths)) + ' images')
X = []
for im_file in tqdm(all_filepaths):
    if im_file == 'none':
        img = np.zeros(desired_im_sz)
    else:
        img = cv2.imread(im_file, cv2.IMREAD_GRAYSCALE)
    try:
        img = cv2.resize(img, dsize=desired_im_sz,
                         interpolation=cv2.INTER_CUBIC)
    except:
        split1 = os.path.splitext(im_file)
        split2 = os.path.splitext(split1[0])
        dummy_file = split2[0] + '.dummy' + split2[1] + split1[1]
        img = cv2.imread(dummy_file, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, dsize=desired_im_sz,
                 interpolation=cv2.INTER_CUBIC)
    img = img.tolist()
    X.append(img)
    del img
    gc.collect()
X = np.asarray(X)

  0%|          | 3/2400 [00:00<02:02, 19.64it/s]

Creating test data: 2400 images


100%|██████████| 2400/2400 [10:19<00:00,  3.87it/s]


In [7]:
X = X.reshape(len(all_filepaths), desired_im_sz[1], desired_im_sz[0], 1) # (len, h, w, ch)
print(X.shape)
if not os.path.exists(DATA_DIR): os.mkdir(DATA_DIR)
hkl.dump(X, os.path.join(DATA_DIR, 'base_model_X_test_debug.hkl'))
hkl.dump(source_list, os.path.join(
    DATA_DIR, 'base_model_sources_test_debug.hkl'))

(2400, 168, 128, 1)
